In [2]:
from opentps.core.io.dataLoader import readData
ImagePath = "TG119_prostate" 
data = readData(ImagePath)
rt_struct = data[0]
ct = data[1]
rt_struct.print_ROINames()
target_name = "PTV"
target = rt_struct.getContourByName(target_name).getBinaryMask(origin=ct.origin,gridSize=ct.gridSize,spacing=ct.spacing)
roi=target


RT Struct UID: 1.2.246.352.71.2.6115.77836.20080227145521
  [0]  BODY
  [1]  PTV
  [2]  Prostate
  [3]  Urinary bladder
  [4]  Rectum


In [ ]:
import copy

import numpy as np
from matplotlib import pyplot as plt
#Import the needed opentps.core packages
import time
import os
import logging
import math
import numpy as np
from matplotlib import pyplot as plt
import sys
from opentps.core.io.dicomIO import writeRTPlan
from opentps.core.processing.planOptimization.tools import evaluateClinical
from opentps.core.data.images import CTImage
from opentps.core.data.images import ROIMask
from opentps.core.data.plan import ObjectivesList
from opentps.core.data.plan import PlanDesign
from opentps.core.data import DVH
from opentps.core.data import Patient
from opentps.core.data.plan import FidObjective
from opentps.core.io.scannerReader import readScanner
from opentps.core.io.serializedObjectIO import saveRTPlan, loadRTPlan
from opentps.core.processing.doseCalculation.doseCalculationConfig import DoseCalculationConfig
from opentps.core.processing.doseCalculation.mcsquareDoseCalculator import MCsquareDoseCalculator
from opentps.core.processing.imageProcessing.resampler3D import resampleImage3DOnImage3D, resampleImage3D
from opentps.core.processing.C_libraries.libRayTracing_wrapper import WET_raytracing
from opentps.core.processing.planOptimization.planOptimization import IMPTPlanOptimizer
from opentps.core.data.plan import PlanDesign
from opentps.core.data import DVH
from opentps.core.io import mcsquareIO
from opentps.core.io.mcsquareIO import _dicomIsocenterToMCsquare
from opentps.core.io.scannerReader import readScanner
from opentps.core.processing.doseCalculation.doseCalculationConfig import DoseCalculationConfig
from opentps.core.processing.doseCalculation.mcsquareDoseCalculator import MCsquareDoseCalculator
from opentps.core.io.dataLoader import readData
from opentps.core.data.plan import ObjectivesList
from opentps.core.data.plan import FidObjective
from opentps.core.io.serializedObjectIO import saveBeamlets, saveRTPlan, loadBeamlets, loadRTPlan
ctCalibration = readScanner(DoseCalculationConfig().scannerFolder)
bdl = mcsquareIO.readBDL(DoseCalculationConfig().bdlFile)
gantryAngles = [90] ##修改Gantry角度
couchAngles = [0]
beamNames = ['beam1']
planDesign = PlanDesign()
planDesign.ct = ct
planDesign.gantryAngles = gantryAngles
planDesign.targetMask = roi
planDesign.beamNames = beamNames
planDesign.couchAngles = couchAngles
planDesign.calibration = ctCalibration
planDesign.spotSpacing = 5.0
planDesign.layerSpacing = 5.0
planDesign.targetMargin = 3.0

plan = planDesign.buildPlan()  # Spot placement
plan.PlanName = "NewPlan"

08/04/2025 11:42:04 AM - opentps.core.data.plan._planDesign - INFO - Building plan ...
08/04/2025 11:42:04 AM - opentps.core.processing.planOptimization.planInitializer - INFO - Target is dilated using a margin of 3.0 mm. This process might take some time.
08/04/2025 11:42:04 AM - opentps.core.processing.imageProcessing.roiMasksProcessing - INFO - Using SITK to dilate mask.
08/04/2025 11:42:19 AM - opentps.core.processing.C_libraries.libRayTracing_wrapper - WARNING - dll found
08/04/2025 11:42:19 AM - opentps.core.processing.C_libraries.libRayTracing_wrapper - WARNING - ready for prepare
08/04/2025 11:42:19 AM - opentps.core.processing.C_libraries.libRayTracing_wrapper - WARNING - ready for call
08/04/2025 11:42:21 AM - opentps.core.data.plan._planDesign - INFO - New plan created in 16.768150568008423 sec
08/04/2025 11:42:21 AM - opentps.core.data.plan._planDesign - INFO - Number of spots: 1378


In [20]:

plan_original=copy.deepcopy(plan)
print(len(plan.beams[0]))
for j in range(len(plan.beams[0])):
    print("layer removed")
    if len(plan.beams[0])==1:
        break
    plan.beams[0].removeLayer(plan.beams[0].layers[0])


16
layer removed
layer removed
layer removed
layer removed
layer removed
layer removed
layer removed
layer removed
layer removed
layer removed
layer removed
layer removed
layer removed
layer removed
layer removed
layer removed


In [ ]:
print(len(plan.beams[0])) #确保只有一个层

1


In [ ]:
plan.beams[0].layers[0].nominalEnergy= 150 ###修改nominalEnergy
plan.beams[0].layers[0]._x = [0.0]
plan.beams[0].layers[0]._y = [0.0]
plan.beams[0].layers[0]._mu = [1.0]
plan.beams[0].layers[0].spotXY

array([[0., 0.]])

In [28]:
print(plan)

Beam
Layer
NominalEnergy: 150
Spots ((x, y), MU): 
(array([0., 0.]), 1.0)


In [29]:
mc2 = MCsquareDoseCalculator()
mc2.beamModel = bdl
mc2.ctCalibration = ctCalibration
mc2.nbPrimaries = 1e6
doseImage = mc2.computeDose(ct,plan)

08/04/2025 11:47:22 AM - opentps.core.processing.doseCalculation.mcsquareDoseCalculator - INFO - Prepare MCsquare Dose calculation
08/04/2025 11:47:22 AM - opentps.core.io.mhdIO - INFO - Write MHD file: C:\Users\gcyyy\openTPS_workspace\Simulations\MCsquare_simulation\CT.mhd
08/04/2025 11:47:22 AM - opentps.core.io.mcsquareIO - INFO - Write plan: C:\Users\gcyyy\openTPS_workspace\Simulations\MCsquare_simulation\PlanPencil.txt
08/04/2025 11:47:23 AM - opentps.core.processing.doseCalculation.mcsquareDoseCalculator - INFO - Start MCsquare simulation


In [ ]:
from ipywidgets import interact, IntSlider
import matplotlib.pyplot as plt
import numpy as np
def plot_Zslice(z_index):
    if not (0 <= z_index < doseImage.imageArray.shape[2]):
     raise ValueError("z_index 超出范围")
    #plt.clf()
    plt.figure(figsize=(8, 6))
    x_size=doseImage.imageArray.shape[0]
    y_size=doseImage.imageArray.shape[1]
    X=np.linspace(0,x_size-1,x_size)
    Y=np.linspace(0,y_size-1,y_size)
    X, Y = np.meshgrid(X, Y)
    #plt.contourf(X, Y, doseImage[:,:,z_index], cmap='jet',levels=10,vmin=0,vmax=np.max(doseImage),alpha=0.6)
    plt.imshow(ct.imageArray[:,:,z_index].transpose(1, 0), cmap='gray', interpolation='nearest', alpha=0.6)
    plt.imshow(doseImage.imageArray[:,:,z_index].transpose(1, 0), cmap='jet', interpolation='nearest', alpha=0.6)
    plt.colorbar()
    #plt.imshow(contourTargetMask.imageArray[:,:,z_index].transpose(1, 0), cmap='binary', alpha=0.3)

    plt.plot()
    plt.show()   
interact(plot_Zslice, z_index=IntSlider(min=0, max=doseImage.imageArray.shape[2]-1, step=1))     

: 

: 